## Fitting Model to Thermomat Data
#### See Labuschagne paper for model

In [1]:
from datahandling import alldatafiles, DataFile, file_parse
from time import time as tm
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
from lmfit import Parameters, minimize, report_fit
from thermomat_functions import cond_model, f2min, find_cut_point, rand_ini_val, parameters
from os import path
from numpy import trapz
import matplotlib.pyplot as plt
from tinydb import TinyDB, Query

C:\Users\Reinhard\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


#### Read files and define parameters

In [2]:
Files = alldatafiles()

In [3]:
db = TinyDB('test_db.json')

In [4]:
db.purge()

In [5]:
db.all()

[]

#### Data Fitting

In [11]:
t = tm()

sample_number_all = []

int_abs_err_all = []
ps_all = []
Q = Query()

with PdfPages('Test.pdf') as pdf:
    for j, f in enumerate(Files[:7]):
        split_tm = tm ()
        
        # Parsing filename
        sample_number = file_parse(f)
        sample_number_all.append(sample_number)
        
        # Check if the relevant data exists and only do fit if necessary
        check = db.search((Q.equipment_name == 'thermomat') & (Q.sample_number == int(sample_number)))
        if len(check) == 0:
        
            # Get data
            time_data, conduct_data = DataFile(f).simple_data()

            # Trim Data
            cut_point = find_cut_point(conduct_data)

            if cut_point != None:
                time_data = time_data[:cut_point]
                conduct_data = conduct_data[:cut_point]

            # Remove offset
            min_cond = min(conduct_data)
            conduct_data[:] = [cond - min_cond for cond in conduct_data]        

            # Find max conductivity to set upper limit of initial beta value
            max_cond = max(conduct_data)
            ini_val_up_lim = [50.0, 500.0, 3.0*max_cond, 1.5]

            # Fit Data with multiple starts 
            starts = 10
            smallest_err = 10000000.0

            for i in range(starts):
                ini_val = rand_ini_val(ini_val_up_lim)
                p = parameters(ini_val, ini_val_up_lim)
                result = minimize(f2min, p, args=(time_data, conduct_data))

                err_list = f2min(p, time_data, conduct_data)
                abs_err_list = map(abs, err_list)
                int_abs_err = trapz(abs_err_list, x=time_data)

                smallest_err = min([smallest_err, int_abs_err])
                if smallest_err == int_abs_err:
                    best_p = p

            int_abs_err_all.append(smallest_err)
            ps_all.append(best_p)

            # Plot fitted model and raw data
            fig = plt.figure()
            fig.suptitle('Sample Number ' + sample_number)
            plt.plot(time_data, conduct_data)
            plt.plot(time_data, cond_model(best_p, time_data))

            pdf.savefig(fig)
            
            # Entering info into tinydb
            p_names = ['theta', 'tau', 'beta', 'm']

            for pn in p_names:
                entry = {'equipment_name': 'thermomat', 
                         'sample_number': int(sample_number),
                         'data_type': pn,
                         'value': best_p[pn].value}
                db.insert(entry)

            db.insert({'equipment_name': 'thermomat', 
                       'sample_number': int(sample_number),
                       'data_type': 'int_of_abs_err',
                       'value': smallest_err})
            
            split_tm = tm() - split_tm
            print 'Completed Fit', (j + 1), 'in', round(split_tm, 2), '(s)'
        else:
            print 'Skipped Fit', (j + 1)

req_time = tm() - t
print '******************'
print 'Time required (min) =', round(req_time/60.0, 2)

Skipped Fit 1
Skipped Fit 2
Skipped Fit 3
Completed Fit 4 in 1.09 (s)
Completed Fit 5 in 9.96 (s)
Completed Fit 6 in 0.37 (s)
Completed Fit 7 in 0.78 (s)
******************
Time required (min) = 0.2


In [12]:
db.all()

[{u'data_type': u'theta',
  u'equipment_name': u'thermomat',
  u'sample_number': 1,
  u'value': 11.0342472296},
 {u'data_type': u'tau',
  u'equipment_name': u'thermomat',
  u'sample_number': 1,
  u'value': 72.4046437512},
 {u'data_type': u'beta',
  u'equipment_name': u'thermomat',
  u'sample_number': 1,
  u'value': 953.6628503484},
 {u'data_type': u'm',
  u'equipment_name': u'thermomat',
  u'sample_number': 1,
  u'value': 0.0229874939},
 {u'data_type': u'int_of_abs_err',
  u'equipment_name': u'thermomat',
  u'sample_number': 1,
  u'value': 63.2704785878},
 {u'data_type': u'theta',
  u'equipment_name': u'thermomat',
  u'sample_number': 10,
  u'value': 3.7118033279000002},
 {u'data_type': u'tau',
  u'equipment_name': u'thermomat',
  u'sample_number': 10,
  u'value': 48.713345303},
 {u'data_type': u'beta',
  u'equipment_name': u'thermomat',
  u'sample_number': 10,
  u'value': 437.9283548757},
 {u'data_type': u'm',
  u'equipment_name': u'thermomat',
  u'sample_number': 10,
  u'value': 0.00